<a href="https://colab.research.google.com/github/nuriddinovN/hogwarts-edai-project/blob/main/AbdullokhonNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models reviewed here:


*   valhalla/t5-base-qg-hl
*   facebook/bart-large-cnn
*   google/flan-t5-base
*   google/mt5-large


1. Installing dependencies and importing libraries

In [2]:
!pip install transformers datasets accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch


2. Load All Three Models & Tokenizers

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Question Generation (valhalla)
qg_model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl").to(device)
qg_tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")

# Summarization (BART)
sum_pipeline = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

# Paraphrasing (FLAN-T5)
para_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
para_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Multilingual Model (mT5)
mt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-large").to(device)
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-large")


Device set to use cpu


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


3. Define Task Functions

Question generation

In [5]:
def generate_question(context, answer):
    input_text = f"generate question: <hl> {answer} <hl> {context}"
    inputs = qg_tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = qg_model.generate(inputs, max_length=64, num_beams=4)
    return qg_tokenizer.decode(outputs[0], skip_special_tokens=True)


Summarization

In [6]:
def summarize_text(text):
    result = sum_pipeline(text, max_length=60, min_length=20, do_sample=False)
    return result[0]['summary_text']


Summarization with multilingual model (mT5)

In [12]:
def mt5_generate(text, task="summarize"):
    if task == "summarize":
        prompt = f"summarize: {text}"
    elif task == "question":
        prompt = f"generate question: {text}"
    else:
        raise ValueError("Only 'summarize' or 'question' supported for mT5.")

    inputs = mt5_tokenizer(prompt, return_tensors="pt", padding=True).to(device)
    outputs = mt5_model.generate(inputs["input_ids"], max_length=64, num_beams=4)
    return mt5_tokenizer.decode(outputs[0], skip_special_tokens=True)


Paraphrasing

In [7]:
def paraphrase_text(text):
    input_text = f"paraphrase: {text}"
    inputs = para_tokenizer(input_text, return_tensors="pt", padding=True).to(device)
    outputs = para_model.generate(inputs["input_ids"], max_length=60, num_beams=4)
    return para_tokenizer.decode(outputs[0], skip_special_tokens=True)


4. Tryng example tasks

In [15]:
# Question Generation
context_qg = "Hogwarts School has Whisper chat, which is publicly open. It is totally anonymous. Students send 1000+ messages like comments, critics, complains and love stories there."
answer_qg = "Hogwarts School"
print("QG Output:", generate_question(context_qg, answer_qg))
print("\n\n")

# Summarization
text_sum = "The development of artificial intelligence (AI) has been one of the most transformative technological advancements in the 21st century. AI systems, particularly those utilizing deep learning and neural networks, have made significant progress in tasks such as image recognition, natural language processing, and decision-making. One of the most prominent applications of AI is in the healthcare sector, where it assists in diagnosing diseases, recommending treatments, and managing patient data more efficiently. AI algorithms are now capable of analyzing medical images with accuracy comparable to human radiologists and are being used to detect conditions such as cancer and neurological disorders at an early stage. In the business world, AI has enabled companies to optimize their operations by automating repetitive tasks, analyzing consumer data to predict market trends, and improving customer service through chatbots and virtual assistants. Financial institutions use AI to detect fraudulent transactions in real-time, while e-commerce platforms leverage recommendation engines powered by AI to personalize the shopping experience for users. Despite these advancements, the rapid integration of AI into daily life raises important ethical and societal questions. Concerns about data privacy, algorithmic bias, and the displacement of human workers are central to ongoing debates. As AI continues to evolve, there is a growing need for frameworks that ensure transparency, accountability, and fairness in its development and deployment. Governments, researchers, and technology companies are actively working on policies and standards to guide the ethical use of AI. These include efforts to make AI more explainable, to audit algorithms for bias, and to ensure that AI systems are aligned with human values. As we look to the future, the challenge will be to harness the immense potential of AI in a way that benefits society as a whole while minimizing harm and inequality."
print("Summary Output:", summarize_text(text_sum))
print("\n\n")


# Paraphrasing
text_para = "If you are a Web Developer, this is a great opportunity to join a team, work on a real project, collaborate with the Hogwarts academic team, and get the chance to participate in the camp from August 4 to 11 with a full scholarship."
print("Paraphrase Output:", paraphrase_text(text_para))
print("\n\n")


# Multilingual summarization (English)
print("mT5 Summary Output:", mt5_generate(
    "The development of artificial intelligence (AI) has been one of the most transformative technological advancements in the 21st century. AI systems, particularly those utilizing deep learning and neural networks, have made significant progress in tasks such as image recognition, natural language processing, and decision-making. One of the most prominent applications of AI is in the healthcare sector, where it assists in diagnosing diseases, recommending treatments, and managing patient data more efficiently. AI algorithms are now capable of analyzing medical images with accuracy comparable to human radiologists and are being used to detect conditions such as cancer and neurological disorders at an early stage. In the business world, AI has enabled companies to optimize their operations by automating repetitive tasks, analyzing consumer data to predict market trends, and improving customer service through chatbots and virtual assistants. Financial institutions use AI to detect fraudulent transactions in real-time, while e-commerce platforms leverage recommendation engines powered by AI to personalize the shopping experience for users. Despite these advancements, the rapid integration of AI into daily life raises important ethical and societal questions. Concerns about data privacy, algorithmic bias, and the displacement of human workers are central to ongoing debates. As AI continues to evolve, there is a growing need for frameworks that ensure transparency, accountability, and fairness in its development and deployment. Governments, researchers, and technology companies are actively working on policies and standards to guide the ethical use of AI. These include efforts to make AI more explainable, to audit algorithms for bias, and to ensure that AI systems are aligned with human values. As we look to the future, the challenge will be to harness the immense potential of AI in a way that benefits society as a whole while minimizing harm and inequality.", task="summarize"))

# Multilingual QG-style prompt (basic)
print("mT5 QG Output:", mt5_generate(
    "Marie Curie discovered radium and polonium. She was the first woman to win a Nobel Prize.", task="question"))

QG Output: What school has Whisper chat?



Summary Output: Happily, this time, we had to change camp dates because of visit of high level people to the Renaissance camp from 4th till 9th. Then, We decided to change Hogwarts camp from 9th till 15th. Today, they announced 1st stage results and I am



Paraphrase Output: If you are a Web Developer, this is a great opportunity to join a team, work on a real project, collaborate with the Hogwarts academic team, and get the chance to participate in the camp from August 4 to 11 with a full scholarship.



mT5 Summary Output: <extra_id_0> and make decisions <extra_id_1> businesses become smarter and smarter. Summary <extra_id_2>: Artificial intelligence has transformed how businesses operate and make decisions <extra_id_3> businesses become smarter and smarter <extra_id_14>s <extra_id_15> businesses <extra_id_16> businesses <extra_id_17> businesses <extra_id_18> businesses <extra_id_19> businesses
mT5 QG Output: <extra_id_0>. She was the firs

5. Comparisons

## Model Comparison Summary

| Task         | Model ID                   | Architecture     | Strengths                              | GPU Use | Speed | Prompt Support | Multilingual |
|--------------|----------------------------|------------------|----------------------------------------|---------|--------|----------------|---------------|
| QG (EN)      | valhalla/t5-base-qg-hl     | T5               | Clean QG, highlight guided             | low      | Fast   | Yes            |  English-only |
| Summarization| facebook/bart-large-cnn    | BART             | Long-text summarization                | mid     | Medium | Yes            |             |
| Paraphrasing | google/flan-t5-base        | T5               | Strong for diverse tasks               | low      | Fast   | Yes            |  Partial     |
| Multi QG/Sum | google/mt5-large           | mT5              | Multilingual generalization            | high   | Slower | Yes (less strict) |  100+ langs |
